## Setup

In [11]:
import numpy as np
import torch
from torchvision import models
from sklearn.model_selection import StratifiedShuffleSplit
from data_utils import *
%load_ext autoreload
%autoreload 2
use_cuda = False #torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print('use cuda = ', use_cuda)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
use cuda =  False


In [2]:
## Load the data
data = load_bolete_data()

ItemsViewHDF5(<HDF5 file "bolete.h5" (mode r)>)


In [3]:
# Get Train and Test Split
X_train, X_test, Y_train, Y_test = get_train_and_test(data, 'bolete-characteristics')


In [ ]:
## Download pretrained model(s)
googlenet = models.googlenet(pretrained=True)
# rex101 = models.resnext101_32x8d(pretrained=True)
# alexnet = models.alexnet(pretrained=True)

## Transfer Learn with cross validation


In [12]:

import torch.optim as optim
import torch.nn as nn
from train import cross_val
import torchvision.transforms as T

dtype = torch.cuda.FloatTensor if use_cuda else torch.float32

def loss_fn(scores, true_y):
    probs = torch.sigmoid(scores)
    if_true = true_y * torch.log(probs)
    if_false = (1-true_y) * torch.log(1 - probs)
    loss = -1 * torch.sum(if_true + if_false)
    return loss

def pred_fn(scores):
    outputs = torch.sigmoid(scores)
    preds = outputs > 0.5
    return preds

num_classes = Y_train.shape[1]

# alexnet.classifier[6] = nn.Linear(4096, num_classes)
# googlenet.fc = nn.Linear(1024, num_classes)

model = googlenet

for param in model.parameters():
    param.requires_grad = False
 
fc_in_feats = model.fc.in_features
model.fc = nn.Linear(fc_in_feats, num_classes)

optimizer = optim.Adam(model.fc.parameters(), betas=(.5, .999), lr=1e-3)

# print(model)

cross_val(
    X_train, 
    Y_train, 
    model, 
    optimizer, 
    nn.MultiLabelSoftMarginLoss(), 
    pred_fn,
    batch_size = 32, 
    num_epochs = 10,
    show_every = 5,
    device = device,
    transform = T.Compose([T.ToTensor()]))

Iter: 0
train Loss: 0.5593 Acc: 27.0214
Iter: 5
val Loss: 0.5681 Acc: 27.0638
Iter: 10
train Loss: 0.4920 Acc: 28.8182
Iter: 15
val Loss: 0.5700 Acc: 26.8830
Iter: 0
train Loss: 0.5159 Acc: 28.1872
Iter: 5
val Loss: 0.4998 Acc: 28.6543


KeyboardInterrupt: 

## Get Species Prediction(s) from Characteristics

In [ ]:
from lookalikes import *